In [24]:
import os
import base64
import json
from pathlib import Path
from typing import List, Dict, Union
from html.parser import HTMLParser
from html import escape, unescape

In [25]:
class ImageEmbedder(HTMLParser):
    def __init__(self, html_path: Path):
        super().__init__()
        self.html_path = html_path
        self.result = ""
        self.success = []
        self.fail = []

    def handle_starttag(self, tag, attrs):
        if tag.lower() == "img":
            attrs_dict = dict(attrs)
            src = attrs_dict.get("src", "")
            img_path = (self.html_path.parent / src).resolve()
            if img_path.exists():
                try:
                    with open(img_path, "rb") as img_file:
                        encoded = base64.b64encode(img_file.read()).decode('utf-8')
                        mime_type = f"image/{img_path.suffix[1:]}"
                        new_src = f"data:{mime_type};base64,{encoded}"
                        attrs_dict["src"] = new_src
                        self.success.append(str(img_path))
                except Exception:
                    self.fail.append(str(img_path))
            else:
                self.fail.append(str(img_path))

            attrs_str = " ".join(f'{k}="{escape(v)}"' for k, v in attrs_dict.items())
            self.result += f"<img {attrs_str}>"
        else:
            attrs_str = " ".join(f'{k}="{escape(v)}"' for k, v in attrs)
            self.result += f"<{tag} {attrs_str}>" if attrs_str else f"<{tag}>"

    def handle_endtag(self, tag):
        self.result += f"</{tag}>"

    def handle_data(self, data):
        self.result += data

    def handle_entityref(self, name):
        self.result += f"&{name};"

    def handle_charref(self, name):
        self.result += f"&#{name};"


In [26]:
class HTMLBatchProcessor:
    def __init__(self, input_paths: List[Union[str, Path]], output_dir: Union[str, Path]):
        self.input_paths = [Path(p) for p in input_paths]
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.results = {
            "success": {},
            "fail": {}
        }

    def get_html_files(self) -> List[Path]:
        html_files = []
        for path in self.input_paths:
            if path.is_file() and path.suffix == ".html":
                html_files.append(path)
            elif path.is_dir():
                html_files.extend(path.rglob("*.html"))  #Explora subdirectorios también
        return html_files

    def process(self) -> Dict[str, Dict[str, List[str]]]:
        html_files = self.get_html_files()
        print(f"Se encontraron {len(html_files)} archivos HTML")

        for html_file in html_files:
            with open(html_file, "r", encoding="utf-8") as f:
                content = f.read()

            parser = ImageEmbedder(html_file)
            parser.feed(content)

            # Guardar nuevo archivo
            output_file = self.output_dir / html_file.name
            with open(output_file, "w", encoding="utf-8") as f_out:
                f_out.write(parser.result)

            self.results["success"][str(html_file)] = parser.success
            self.results["fail"][str(html_file)] = parser.fail

        return self.results

In [27]:

# Ejecución
if __name__ == "__main__":
    rutas = ["/content/drive/MyDrive/Prueba_tecnica_tuya/htmls_prueba"]
    salida = "/content/drive/MyDrive/Prueba_tecnica_tuya/html_modificados"

    processor = HTMLBatchProcessor(rutas, salida)
    resultado = processor.process()

    total_ok = sum(len(v) for v in resultado["success"].values())
    total_fail = sum(len(v) for v in resultado["fail"].values())
    print(f"\n Imágenes procesadas correctamente: {total_ok}")
    print(f"❌ Imágenes con error: {total_fail}")

    resumen_path = "/content/drive/MyDrive/Prueba_tecnica_tuya/resumen_resultado.json"
    with open(resumen_path, "w", encoding="utf-8") as f:
        json.dump(resultado, f, indent=4, ensure_ascii=False)
    print(f"\n Resumen guardado en: {resumen_path}")

Se encontraron 3 archivos HTML

 Imágenes procesadas correctamente: 9
❌ Imágenes con error: 0

 Resumen guardado en: /content/drive/MyDrive/Prueba_tecnica_tuya/resumen_resultado.json


In [ ]:
# Crear README.md automáticamente
readme_content = """
# Prueba Técnica - Tuya

Este repositorio contiene la solución al primer ejercicio de la prueba técnica para Tuya.

## Enunciado

Procesar archivos HTML con imágenes externas, convertirlas a base64, reemplazarlas en el HTML y generar un archivo de resumen con los resultados.

## Tecnologías

- Python 3.10+
- Librerías estándar: `os`, `base64`, `json`, `html.parser`, `pathlib`

## Archivos

- `procesamiento_html.py`: Código principal para procesar HTMLs
- `resumen_resultado.json`: Informe con imágenes embebidas exitosamente o con error
- `README.md`: Este documento

## Ejecución

```python
# Ejecuta desde Colab o como script
python procesamiento_html.py
